In [ ]:
import tensorflow as tf
from algomorphism.model.models import GAE
from datasets import GaeDataset
from algomorphism.figure.opt import multiple_models_history_figure
from algomorphism.method.graph import a2g
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
import os

In [ ]:
# optional for GPU usage

# for gpu in tf.config.list_physical_devices('GPU'):
#     print(gpu)
#     tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
gaed = GaeDataset()

In [ ]:
for x, atld, a in gaed.train:
    A = a
    X = x
    Atld = atld

In [ ]:
w_p = float(A.shape[0]*A.shape[1] * A.shape[1] - tf.reduce_sum(A)) / tf.reduce_sum(A)
norm = A.shape[1] * A.shape[2] / float((A.shape[1] * A.shape[2] - tf.reduce_sum(A)) * 2)

df_list = [A.shape[1], 32, 64]

adam = tf.keras.optimizers.Adam()
gae = GAE(gaed, df_list ,w_p, norm=norm, optimizer=adam, learning_rate=1e-4, ip_weights=True)

In [ ]:
gae.set_lr_rate(1e-2)
gae.set_clip_norm(0.0)
gae.train(gaed, 200)

In [ ]:
ahat = tf.nn.sigmoid(gae([X,Atld])[0]).numpy()[0]
plt.imshow(ahat, vmin=0, vmax=1)
ahat

In [ ]:
g = a2g((ahat>0.5).astype(int))
nx.draw(g, with_labels=True,font_size=20,node_size=600)

In [ ]:
if any(gae.history.values()):
    multiple_models_history_figure([gae])

In [ ]:
idx_save = "64"
idx_load = ""

In [ ]:
Labels = ['0', '1', '2', '3', '4', '5']
emb = gae.encoder(X,Atld)
emb = emb[0]
emb_dict = dict(zip(Labels,emb.numpy()))
emb.shape

In [ ]:
# np.save("./data/gae/gae-node-embs{}".format(idx_save), emb_dict)

In [ ]:
pca = PCA(n_components=2)
pca_emb = pca.fit_transform(emb)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(pca_emb[:,0], pca_emb[:,1],'o', label='node embeddings', markersize=15)

for (v,l) in zip(pca_emb, Labels):
    plt.text(v[0],v[1],l,fontsize=20)
plt.legend()

# save fig
# if not os.path.exists('./DataFigures/gae/'):
#         os.makedirs('./DataFigures/gae/')
# plt.savefig('DataFigures/gae/node-embs.eps', format='eps')